In [1]:
from vpl_extraction.vpl_star import VPLStar
from examples.dyck1 import Dyck1

In [2]:
dyck1_grammar = Dyck1()

vpl_star = VPLStar(dyck1_grammar)

In [3]:
result = vpl_star.learn()

In [4]:
print(str(result.tree_automaton))

TreeAutomata( 
 -- states=['ε []', ') [") [\'ε []\']"]', ") ['ε []']"] 
 -- final_states=['ε []'] 
 -- transitions=['Key(), [") [\'ε []\']"])', 'Key(ε, [])', 'Key((, [\'ε []\', ") [\'ε []\']"])', 'Key((, [") [\'ε []\']", ") [\'ε []\']"])', "Key(), ['ε []'])", 'Key((, [") [\'ε []\']", \'ε []\'])'] 
)


In [5]:
from models.vpg import vpg_from_tree_automata

vpg = vpg_from_tree_automata(result.tree_automaton, dyck1_grammar.alphabet)

print(vpg.print_grammar())

Grammar VPG
Variables: q1, q0, q2
Start Symbols: ε []
Rules:
q0 -> ε    # epsilon
q0 -> ( q0 ) q0    # push-pop
q2 -> ( q2 ) q0    # push-pop


Variable Map:
ε [] -> q0
) [") ['ε []']"] -> q1
) ['ε []'] -> q2



In [6]:
print(len(result.tree_automaton.states))

3


In [7]:
for k, v in result.tree_automaton.transitions.items():
    print(f"{k} - Val {v}")

Key(), [") ['ε []']"]) - Val ) [") ['ε []']"]
Key(ε, []) - Val ε []
Key((, ['ε []', ") ['ε []']"]) - Val ε []
Key((, [") ['ε []']", ") ['ε []']"]) - Val ) ['ε []']
Key(), ['ε []']) - Val ) ['ε []']
Key((, [") ['ε []']", 'ε []']) - Val ε []


In [8]:
random_word_count = 10000
counterexample = False
for _ in range(random_word_count):
    word = dyck1_grammar.get_random_word()
    
    if result.is_accepted(word) != dyck1_grammar.is_accepted(word):
        print(f"VPLStar does not match Dyck1 for the word: {word}")
        counterexample = True
        break

if not counterexample:
    print("VPLStar matches Dyck1 for all tested words.")

VPLStar matches Dyck1 for all tested words.


In [5]:
import torch
import json
from base.vpl import VPL
from transformer_checker.transformer import (
    TransformerClassifier,
    TransformerClassifierConfig
)

class TransformerWrapper(VPL):
    def __init__(self, metadata_path: str, tokenizer, alphabet):
        self.tokenizer = tokenizer
        self.alphabet = alphabet

        self.metadata = self._load_metadata(metadata_path)
        self.model = self._load_model(self.metadata)

    
    def _load_metadata(self, metadata_path: str) -> dict:
        # Load metadata from the specified path
        with open(metadata_path, 'r') as file:
            metadata = json.loads(file.read())

        return metadata

    
    def _load_model(self, metadata: dict) -> TransformerClassifier:
        config = metadata.get('model_config', None)

        if config is None:
            raise ValueError("Metadata does not contain 'config' key.")

        model_config = TransformerClassifierConfig(
            **config
        )

        model = TransformerClassifier(
            config=model_config
        )

        model_weights_path = metadata.get('wheigts_path', None)
        if model_weights_path is None:
            raise ValueError("Metadata does not contain 'weights_path' key.")

        model_weights = torch.load(model_weights_path)
        model.load_state_dict(model_weights)
        return model
    
    
    def is_accepted(self, sequence: str) -> bool:
        return self.model(sequence)
    

/Users/juanpe/miniconda3/envs/TransformerExtraction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
t = TransformerWrapper(
    metadata_path='./experiments/models/metadata/transformer_model.json',
    tokenizer=None,  # Replace with actual tokenizer if needed
    alphabet=['(', ')']  # Replace with actual alphabet if needed
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: juancommits. Use `wandb login --relogin` to force relogin


RuntimeError: Error(s) in loading state_dict for TransformerClassifier:
	size mismatch for embedding.weight: copying a param with shape torch.Size([5, 256]) from checkpoint, the shape in current model is torch.Size([8, 256]).